<a href="https://colab.research.google.com/github/LokeshVadlamudi/MachineLearningClass/blob/master/AlternusVeraLoki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Title vs Body** factor

In [0]:
import pandas as pd
import numpy as np

Taking Liar Liar dataset

In [0]:
train_data = '/content/drive/My Drive/LiarDataset/train.tsv'
test_data = '/content/drive/My Drive/LiarDataset/test.tsv'
valid_data = '/content/drive/My Drive/LiarDataset/valid.tsv'

In [0]:
df = pd.read_csv(train_data,sep='\t')

In [5]:
df

,2635.json,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0.1,0.2,0.3,a mailer
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10234,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report"""
10235,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview
10236,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate
10237,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...


mentioning the column names

In [0]:
cols = ['jsonid', 'label', 'headline_text', 'subject', 'speaker', 
            'speakerjobtitle', 'stateinfo','partyaffiliation', 
            'barelytruecounts', 'falsecounts','halftruecounts',
            'mostlytrueocounts','pantsonfirecounts','context']

In [0]:
headerBody = 'headline_text'
labelEncoded = 'labelEncoded'
finalHeaderBody = 'finalHeaderBody' 

Convert multiple class labels into 2 categories, 1 or 0

In [0]:
t_labels = ['mostly-true','half-true','original','true']
def convert_Label(i):
  if i not in t_labels:
    return 0
  else:
    return 1

creating the dataframes

In [0]:
df_train = pd.read_csv(train_data, sep='\t', names = cols)
df_test = pd.read_csv(test_data, sep='\t', names = cols)
df_valid = pd.read_csv(valid_data, sep='\t', names = cols)

In [10]:
df_train.head()

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocounts,pantsonfirecounts,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [11]:
df_train.shape

(10240, 14)

converting the labels row by row and storing in new labelencoded column

In [0]:
df_train[labelEncoded] = df_train.apply(lambda row: convert_Label(row['label']), axis=1)
df_test[labelEncoded] = df_test.apply(lambda row: convert_Label(row['label']), axis=1)
df_valid[labelEncoded] = df_valid.apply(lambda row: convert_Label(row['label']), axis=1)

In [13]:
df_train.head()

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocounts,pantsonfirecounts,context,labelEncoded
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,0
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,1
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,1
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,0
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,1


In [14]:
df_train.shape

(10240, 15)

importing nltk libs


In [15]:
import string
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
stemmer = SnowballStemmer('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


function for cleaning the data

In [0]:
def clean_data(data):
  sw = stopwords.words('english')
  data = [w.lower() for w in data.split() if (w.lower() not in sw) and (len(w)>3)]
  data = " ".join(data)
  data = stemmer.stem(WordNetLemmatizer().lemmatize(data, pos='v'))
  translator = str.maketrans('', '', string.punctuation)
  return data.translate(translator)

In [17]:
clean_data(df_train['headline_text'][0])

'says annies list political group supports thirdtrimester abortions demand'

using the function to clean data in the header body on all the rows in train dataframe

In [0]:
df_train[finalHeaderBody] = df_train.apply(lambda row: clean_data(row[headerBody]), axis=1)
df_test[finalHeaderBody] = df_test.apply(lambda row: clean_data(row[headerBody]), axis=1)
df_valid[finalHeaderBody] = df_valid.apply(lambda row: clean_data(row[headerBody]), axis=1)

In [19]:
df_train.shape

(10240, 16)

saving the dataframe to a csv


In [0]:
df_train.to_csv("/content/drive/My Drive/LiarDataset/train_cleantext.csv", sep=',')
df_test.to_csv("/content/drive/My Drive/LiarDataset/test_cleantext.csv", sep=',')
df_valid.to_csv("/content/drive/My Drive/LiarDataset/valid_cleantext.csv", sep=',')

In [0]:
df_train_clean = pd.read_csv('/content/drive/My Drive/LiarDataset/train_cleantext.csv')

In [22]:
df_train_clean

,Unnamed: 0,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocounts,pantsonfirecounts,context,labelEncoded,finalHeaderBody
0,0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,0,says annies list political group supports thir...
1,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,1,decline coal start started natural took starte...
2,2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,1,hillary clinton agrees john mccain voting give...
3,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,0,health care reform legislation likely mandate ...
4,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,1,economic turnaround started term
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,10235,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report""",1,larger number shark attacks florida cases vote...
10236,10236,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview,1,democrats become party atlanta metro area blacks
10237,10237,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate,1,says alternative social security operates galv...
10238,10238,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...,0,lifting us cuban embargo allowing travel cuba


we just take three columns that are needed for our factor

In [0]:
df_train_process = df_train_clean[['subject','finalHeaderBody','labelEncoded']]

In [24]:
df_train_process

,subject,finalHeaderBody,labelEncoded
0,abortion,says annies list political group supports thir...,0
1,"energy,history,job-accomplishments",decline coal start started natural took starte...,1
2,foreign-policy,hillary clinton agrees john mccain voting give...,1
3,health-care,health care reform legislation likely mandate ...,0
4,"economy,jobs",economic turnaround started term,1
...,...,...,...
10235,"animals,elections",larger number shark attacks florida cases vote...,1
10236,elections,democrats become party atlanta metro area blacks,1
10237,"retirement,social-security",says alternative social security operates galv...,1
10238,"florida,foreign-policy",lifting us cuban embargo allowing travel cuba,0


**factor**

In [25]:
!pip install nltk
!pip install gensim
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
import nltk
import gensim
from gensim.models.doc2vec import TaggedDocument
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models.doc2vec import Doc2Vec
from wordcloud import WordCloud 
import matplotlib.pyplot as plt
import urllib
% matplotlib inline

create word tags on all the body texts


In [0]:
tagged_data = [TaggedDocument(words=article.split(),tags=[str(index)]) for index, article in enumerate(df_train_process['finalHeaderBody'])]

In [28]:
tagged_data

[TaggedDocument(words=['says', 'annies', 'list', 'political', 'group', 'supports', 'thirdtrimester', 'abortions', 'demand'], tags=['0']),
 TaggedDocument(words=['decline', 'coal', 'start', 'started', 'natural', 'took', 'started', 'begin', 'president', 'george', 'bushs', 'administration'], tags=['1']),
 TaggedDocument(words=['hillary', 'clinton', 'agrees', 'john', 'mccain', 'voting', 'give', 'george', 'bush', 'benefit', 'doubt', 'iran'], tags=['2']),
 TaggedDocument(words=['health', 'care', 'reform', 'legislation', 'likely', 'mandate', 'free', 'change', 'surgeries'], tags=['3']),
 TaggedDocument(words=['economic', 'turnaround', 'started', 'term'], tags=['4']),
 TaggedDocument(words=['chicago', 'bears', 'starting', 'quarterbacks', 'last', 'years', 'total', 'number', 'tenured', 'uw', 'faculty', 'fired', 'last', 'decades'], tags=['5']),
 TaggedDocument(words=['dunnam', 'lived', 'district', 'represents', 'years', 'now'], tags=['6']),
 TaggedDocument(words=['person', 'stage', 'worked', 'acti

doc 2 vectorizer model is created based on above tagged texts

In [29]:
model = Doc2Vec()
model.build_vocab(tagged_data)
model.train(tagged_data,total_examples=model.corpus_count,epochs=model.iter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


the break_subject function cleans the subject and creates a list of words in it

In [0]:
def break_subject(text):
  if type(text) is float:
    return []
  import re
  return re.split('; |, |-',text)

In [117]:
break_subject('job accomplishments')

['job', 'accomplishments']

applying the above function on subject col to break it into final_subject col


In [71]:
df_train_process['final_subject'] = df_train_process['subject'].apply(lambda x: break_subject(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [72]:
df_train_process

,subject,finalHeaderBody,labelEncoded,similar_doc,subject_synonyms,final_subject
0,abortion,says annies list political group supports thir...,0,7780,"[abortion, miscarriage, abortion]",[abortion]
1,"energy,history,job-accomplishments",decline coal start started natural took starte...,1,4786,[],"[energy,history,job, accomplishments]"
2,foreign-policy,hillary clinton agrees john mccain voting give...,1,4786,[],"[foreign, policy]"
3,health-care,health care reform legislation likely mandate ...,0,4786,[],"[health, care]"
4,"economy,jobs",economic turnaround started term,1,4786,[],"[economy,jobs]"
...,...,...,...,...,...,...
10235,"animals,elections",larger number shark attacks florida cases vote...,1,4786,[],"[animals,elections]"
10236,elections,democrats become party atlanta metro area blacks,1,4437,"[election, election, election, election]",[elections]
10237,"retirement,social-security",says alternative social security operates galv...,1,4786,[],"[retirement,social, security]"
10238,"florida,foreign-policy",lifting us cuban embargo allowing travel cuba,0,4786,[],"[florida,foreign, policy]"


we create a function for generating the synonyms for a word below

In [0]:

from nltk.corpus import wordnet 
def generate_Synonyms(wordlist):
    synonyms = []
    for i in wordlist:
      for syn in wordnet.synsets(i): 
          for l in syn.lemmas(): 
              synonyms.append(l.name())
    return(synonyms) 

In [75]:
generate_Synonyms(['energy','history','job', 'accomplishments'])

['energy',
 'free_energy',
 'energy',
 'vigor',
 'vigour',
 'zip',
 'energy',
 'push',
 'get-up-and-go',
 'energy',
 'muscularity',
 'vigor',
 'vigour',
 'vim',
 'energy',
 'vim',
 'vitality',
 'energy',
 'Department_of_Energy',
 'Energy_Department',
 'Energy',
 'DOE',
 'history',
 'history',
 'account',
 'chronicle',
 'story',
 'history',
 'history',
 'history',
 'occupation',
 'business',
 'job',
 'line_of_work',
 'line',
 'job',
 'task',
 'chore',
 'job',
 'job',
 'job',
 'job',
 'job',
 'problem',
 'job',
 'Job',
 'Job',
 'job',
 'Job',
 'Book_of_Job',
 'caper',
 'job',
 'job',
 'subcontract',
 'farm_out',
 'job',
 'job',
 'speculate',
 'job',
 'accomplishment',
 'achievement',
 'skill',
 'accomplishment',
 'acquirement',
 'acquisition',
 'attainment']

In [0]:
# df_train_process['subject_synonyms'] = df_train_process.apply(lambda row: df_train_process(row['subject']), axis=1)

we apply the function above to the final subject col to generate multiple synonyms for our subject.


In [77]:
df_train_process['subject_synonyms'] = df_train_process['final_subject'].apply(lambda x: generate_Synonyms(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
df_train_process

,subject,finalHeaderBody,labelEncoded,similar_doc,subject_synonyms,final_subject
0,abortion,says annies list political group supports thir...,0,7780,"[abortion, miscarriage, abortion]",[abortion]
1,"energy,history,job-accomplishments",decline coal start started natural took starte...,1,4786,"[accomplishment, achievement, skill, accomplis...","[energy,history,job, accomplishments]"
2,foreign-policy,hillary clinton agrees john mccain voting give...,1,4786,"[foreign, foreign, strange, alien, foreign, ex...","[foreign, policy]"
3,health-care,health care reform legislation likely mandate ...,0,4786,"[health, wellness, health, care, attention, ai...","[health, care]"
4,"economy,jobs",economic turnaround started term,1,4786,[],"[economy,jobs]"
...,...,...,...,...,...,...
10235,"animals,elections",larger number shark attacks florida cases vote...,1,4786,[],"[animals,elections]"
10236,elections,democrats become party atlanta metro area blacks,1,4437,"[election, election, election, election]",[elections]
10237,"retirement,social-security",says alternative social security operates galv...,1,4786,"[security, security, protection, security, sec...","[retirement,social, security]"
10238,"florida,foreign-policy",lifting us cuban embargo allowing travel cuba,0,4786,"[policy, policy, policy, insurance_policy, ins...","[florida,foreign, policy]"


now the below function gets the similarity score between our final header body and the subject synonyms cols.

In [0]:
def get_similarity(statement):
  test = re.sub(r"\W", " ",str(statement))
  test =re.sub(r'[,\.!?]', " ",str(test))
  test = re.sub(r" \d+", " ",str(test))
  test= word_tokenize(test.lower())

  infered_vector = model.infer_vector(test)
  doc_number , similarity_percentage = model.docvecs.most_similar([infered_vector], topn = 1)[0]

  return doc_number

In [79]:
df_train_process['similar_doc'] = df_train_process['subject_synonyms'].apply(lambda x: get_similarity(x))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


The similar_doc col holds the similarity score of our subject and body.

In [80]:
df_train_process

,subject,finalHeaderBody,labelEncoded,similar_doc,subject_synonyms,final_subject
0,abortion,says annies list political group supports thir...,0,7780,"[abortion, miscarriage, abortion]",[abortion]
1,"energy,history,job-accomplishments",decline coal start started natural took starte...,1,4147,"[accomplishment, achievement, skill, accomplis...","[energy,history,job, accomplishments]"
2,foreign-policy,hillary clinton agrees john mccain voting give...,1,9015,"[foreign, foreign, strange, alien, foreign, ex...","[foreign, policy]"
3,health-care,health care reform legislation likely mandate ...,0,4893,"[health, wellness, health, care, attention, ai...","[health, care]"
4,"economy,jobs",economic turnaround started term,1,4786,[],"[economy,jobs]"
...,...,...,...,...,...,...
10235,"animals,elections",larger number shark attacks florida cases vote...,1,4786,[],"[animals,elections]"
10236,elections,democrats become party atlanta metro area blacks,1,5503,"[election, election, election, election]",[elections]
10237,"retirement,social-security",says alternative social security operates galv...,1,2961,"[security, security, protection, security, sec...","[retirement,social, security]"
10238,"florida,foreign-policy",lifting us cuban embargo allowing travel cuba,0,3238,"[policy, policy, policy, insurance_policy, ins...","[florida,foreign, policy]"


In [83]:
model.most_similar('obama')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('president', 0.9995503425598145),
 ('obamas', 0.9989464282989502),
 ('barack', 0.9974798560142517),
 ('bush', 0.9974783658981323),
 ('says', 0.9974116086959839),
 ('promised', 0.9973248243331909),
 ('said', 0.9972907900810242),
 ('became', 0.9969698786735535),
 ('george', 0.9969490766525269),
 ('office', 0.996861457824707)]

**adding fake new dataset**

In [0]:
train_fake_data = '/content/drive/My Drive/FakeNewsDataset/train.csv'

In [0]:
df_train_fake = pd.read_csv(train_fake_data)

In [89]:
df_train_fake['text'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [90]:
df_train_fake

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [0]:
def clean_data_fake(statement):
  sw = stopwords.words('english')
  test = re.sub(r"\W", " ",str(statement))
  test =re.sub(r'[,\.!?]', " ",str(test))
  test = re.sub(r" \d+", " ",str(test))
  # d1 = re.split('; |, |\*|\n| |:',data)
  # # d1 = []

  # for w in data.split():
  #   if type(w) is not float:
  #     continue
  #   else:
  #     if w.lower() not in sw and (len(w)>3):
  #       d1.append(w.lower())

  # # data = [w.lower() for w in data.split() if (w.lower() not in sw) and (len(w)>3)]
  # data = " ".join(d1)
  # data = stemmer.stem(WordNetLemmatizer().lemmatize(data, pos='v'))
  # translator = str.maketrans('', '', string.punctuation)
  # return data.translate(translator)
  return test

In [0]:
df_train_fake['final_subject'] = df_train_fake.apply(lambda row: clean_data_fake(row['title']), axis=1)
df_train_fake['finalHeaderBody'] = df_train_fake.apply(lambda row: clean_data_fake(row['text']), axis=1)

In [111]:
df_train_fake

,id,title,author,text,label,final_subject,finalHeaderBody
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide We Didn t Even See Comey s Let...,House Dem Aide We Didn t Even See Comey s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,FLYNN Hillary Clinton Big Woman on Campus ...,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired,Why the Truth Might Get You Fired October ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...,Videos Civilians Killed In Single US Airstrik...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...,Print An Iranian woman has been sentenced to ...
...,...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Rapper T I Trump a Poster Child For White S...,Rapper T I unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,N F L Playoffs Schedule Matchups and Odds ...,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Macy s Is Said to Receive Takeover Approach by...,The Macy s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,NATO Russia To Hold Parallel Exercises In Bal...,NATO Russia To Hold Parallel Exercises In Bal...


In [0]:
# df_train_fake['final_subject'] = df_train_fake.apply(lambda row: clean_data_fake(row['title']), axis=1)
df_train_fake['final_subject'] = df_train_fake['finalHeaderBody'].apply(lambda x: break_subject(x))

In [135]:
df_train_fake

,id,title,author,text,label,final_subject,finalHeaderBody
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,[House Dem Aide We Didn t Even See Comey s Le...,House Dem Aide We Didn t Even See Comey s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,[Ever get the feeling your life circles the ro...,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,[Why the Truth Might Get You Fired October ...,Why the Truth Might Get You Fired October ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,[Videos Civilians Killed In Single US Airstri...,Videos Civilians Killed In Single US Airstrik...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,[Print An Iranian woman has been sentenced to...,Print An Iranian woman has been sentenced to ...
...,...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,[Rapper T I unloaded on black celebrities wh...,Rapper T I unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,[When the Green Bay Packers lost to the Washin...,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,[The Macy s of today grew from the union of se...,The Macy s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,[NATO Russia To Hold Parallel Exercises In Ba...,NATO Russia To Hold Parallel Exercises In Bal...


In [0]:
def change_final_fake(text):
    for i in text:
      d = i.split()
    return d

In [0]:
df_train_fake['final_subject'] = df_train_fake['final_subject'].apply(lambda x: change_final_fake(x))

In [138]:
df_train_fake

,id,title,author,text,label,final_subject,finalHeaderBody
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,"[House, Dem, Aide, We, Didn, t, Even, See, Com...",House Dem Aide We Didn t Even See Comey s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"[Ever, get, the, feeling, your, life, circles,...",Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,"[Why, the, Truth, Might, Get, You, Fired, Octo...",Why the Truth Might Get You Fired October ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,"[Videos, Civilians, Killed, In, Single, US, Ai...",Videos Civilians Killed In Single US Airstrik...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,"[Print, An, Iranian, woman, has, been, sentenc...",Print An Iranian woman has been sentenced to ...
...,...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,"[Rapper, T, I, unloaded, on, black, celebritie...",Rapper T I unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"[When, the, Green, Bay, Packers, lost, to, the...",When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,"[The, Macy, s, of, today, grew, from, the, uni...",The Macy s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"[NATO, Russia, To, Hold, Parallel, Exercises, ...",NATO Russia To Hold Parallel Exercises In Bal...


In [0]:
df_train_fake['subject_synonyms'] = df_train_fake['final_subject'].apply(lambda x: generate_Synonyms(x))

In [140]:
df_train_fake

,id,title,author,text,label,final_subject,finalHeaderBody,subject_synonyms
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,"[House, Dem, Aide, We, Didn, t, Even, See, Com...",House Dem Aide We Didn t Even See Comey s Let...,"[house, firm, house, business_firm, house, hou..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"[Ever, get, the, feeling, your, life, circles,...",Ever get the feeling your life circles the rou...,"[ever, of_all_time, always, ever, e'er, ever, ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,"[Why, the, Truth, Might, Get, You, Fired, Octo...",Why the Truth Might Get You Fired October ...,"[why, wherefore, truth, truth, the_true, verit..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,"[Videos, Civilians, Killed, In, Single, US, Ai...",Videos Civilians Killed In Single US Airstrik...,"[video, picture, video_recording, video, video..."
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,"[Print, An, Iranian, woman, has, been, sentenc...",Print An Iranian woman has been sentenced to ...,"[print, print, mark, print, print, print, prin..."
...,...,...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,"[Rapper, T, I, unloaded, on, black, celebritie...",Rapper T I unloaded on black celebrities who...,"[rapper, knocker, doorknocker, rapper, thymine..."
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"[When, the, Green, Bay, Packers, lost, to, the...",When the Green Bay Packers lost to the Washing...,"[green, greenness, viridity, park, commons, co..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,"[The, Macy, s, of, today, grew, from, the, uni...",The Macy s of today grew from the union of sev...,"[second, sec, s, sulfur, S, sulphur, atomic_nu..."
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"[NATO, Russia, To, Hold, Parallel, Exercises, ...",NATO Russia To Hold Parallel Exercises In Bal...,"[North_Atlantic_Treaty_Organization, NATO, Sov..."
